## Settings

In [1]:
import sklearn
print(sklearn.__version__)

1.6.0


In [20]:
import os
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler

os.getcwd()
version = 'Combined_Range2'
raw_data_path = 'combined_Range2.csv'
basic_data_path = f'{version}_basic_data.csv'
normalized_data_path = f'{version}_normalized_data.csv'

raw_data = pd.read_csv(raw_data_path)

In [21]:
# raw_data_list = []
# for percent in [2, 3]:
#     raw_data_path = f'basic_info_df_modified_PlusMinus{percent}.csv'
#     df = pd.read_csv(raw_data_path)
#     df["예가범위"] = percent
#     raw_data_list.append(df)
# raw_data = pd.concat(raw_data_list, ignore_index=True)
raw_data = pd.read_csv(raw_data_path)

# 1. 필요한 함수 모음, 실행

In [22]:
# 원하는 컬럼 순서 정의
desired_columns = [
    '공고번호',
    '기초금액',
    'A값',
    '투찰률(%)',
    '참여업체수',
    '예가범위',
    '공고구분표시',
    '공고제목',
    '발주처(수요기관)',
    '지역제한'
]

# DataFrame 재구성
def reorganize_dataframe(df, desired_columns):
    """
    DataFrame의 컬럼을 원하는 순서로 재배열하고, 
    숫자 컬럼들은 그대로 유지합니다.
    """
    # 12번째 이후의 컬럼들 가져오기
    remaining_columns = [col for col in df.columns if col not in desired_columns and col.count('_') == 1 and len(col.split('_')[0]) == 3 and len(col.split('_')[1]) == 3]
    
    # 원하는 순서의 컬럼과 나머지 컬럼들 합치기
    final_columns = desired_columns + remaining_columns
    # 새로운 순서로 DataFrame 재구성
    reorganized_df = df[final_columns]
    return reorganized_df

def normalize_and_transform_data(df):
    """
    DataFrame의 특정 컬럼들을 변환하고 정규화합니다.
    유효하지 않은 투찰률 데이터를 가진 행은 제거합니다.
    """
    import numpy as np
    from sklearn.preprocessing import StandardScaler

    # 결과 DataFrame 복사
    result_df = df.copy()
    
    # 투찰률 데이터 전처리
    # 1. '-' 값을 NaN으로 변환
    result_df['투찰률(%)'] = result_df['투찰률(%)'].replace('-', np.nan)
    # 2. 숫자로 변환 시도 (실패하면 NaN)
    result_df['투찰률(%)'] = pd.to_numeric(result_df['투찰률(%)'], errors='coerce')
    # 3. NaN 값을 가진 행 제거
    valid_rows = result_df['투찰률(%)'].notna()
    result_df = result_df[valid_rows]
    
    print(f"Removed {(~valid_rows).sum()} rows with invalid '투찰률(%)' values")
    
    # 스케일러 초기화
    scaler = StandardScaler()
    
    # 1. 기초금액 변환
    result_df['log_기초금액'] = np.log1p(result_df['기초금액'].astype(float))
    result_df['log_기초금액_norm'] = scaler.fit_transform(result_df[['log_기초금액']])
    result_df['기초금액_norm'] = scaler.fit_transform(result_df[['기초금액']])
    
    # 2. A값/기초금액 변환
    result_df['A값/기초금액_norm'] = result_df['A값'].astype(float) / (1+result_df['기초금액'].astype(float))
    result_df['A값/기초금액_norm'] = scaler.fit_transform(result_df[['A값/기초금액_norm']])
    
    # 3. 투찰률 정규화 (이제 모든 값이 유효한 숫자)
    result_df['투찰률_norm'] = scaler.fit_transform(result_df[['투찰률(%)']])
    
    # 4. 참여업체수 변환
    result_df['log_참여업체수'] = np.log1p(1+ result_df['참여업체수'].astype(float))
    result_df['log_참여업체수_norm'] = scaler.fit_transform(result_df[['log_참여업체수']])
    result_df['참여업체수_norm'] = scaler.fit_transform(result_df[['참여업체수']])

    # 5. 참여업체수/기초금액 변환
    num_colums = [col for col in result_df.columns if col.count('_') == 1 and len(col.split('_')[0]) == 3 and len(col.split('_')[1]) == 3]
    for col in num_colums:
       result_df[col] = result_df[col] / result_df['참여업체수']
    
    # 컬럼 순서 설정
    keep_columns = [
        '공고번호',
        '기초금액',
        'log_기초금액',
        'log_기초금액_norm',
        '기초금액_norm',
        'A값',
        'A값/기초금액_norm',
        '투찰률(%)',
        '투찰률_norm',
        '참여업체수',
        'log_참여업체수',
        'log_참여업체수_norm',
        '예가범위',
        '공고구분표시',
        '공고제목',
        '발주처(수요기관)',
        '지역제한',
    ]
    
    final_columns = keep_columns + num_colums
    result_df = result_df[final_columns]

    return result_df
    return result_df

In [23]:
# DataFrame 재구성 실행
raw_data_reorganized = reorganize_dataframe(raw_data, desired_columns)
# normalize 실행
transformed_df = normalize_and_transform_data(raw_data_reorganized)
# CSV 파일로 저장
transformed_df.to_csv(normalized_data_path, index=False)

Removed 4 rows with invalid '투찰률(%)' values


# Pipeline 함수 정의

In [24]:
def process_pipeline(df, desired_columns):
    # reorganize data
    organized_df = reorganize_dataframe(df, desired_columns)
    # normalize data
    normalized_df = normalize_and_transform_data(organized_df)
    return organized_df, normalized_df
organized_df, normalized_df = process_pipeline(raw_data, desired_columns)

# save data
# organized_data_path = 'Basic_data_agg_v4.csv'
# normalized_data_path = 'Modified_Basic_data_agg_v4.csv'
# # organized_df.to_csv(basic_data_path, index=False)
# organized_df.to_csv(organized_data_path, index=False)
# normalized_df.to_csv(normalized_data_path, index=False)
# shuffled_df = shuffle_specific_columns(normalized_df)
# shuffled_df.to_csv(f'Shuffled_Basic_data_agg_v4.csv', index=False)

Removed 4 rows with invalid '투찰률(%)' values


In [25]:
normalized_df.columns[:30]

Index(['공고번호', '기초금액', 'log_기초금액', 'log_기초금액_norm', '기초금액_norm', 'A값',
       'A값/기초금액_norm', '투찰률(%)', '투찰률_norm', '참여업체수', 'log_참여업체수',
       'log_참여업체수_norm', '예가범위', '공고구분표시', '공고제목', '발주처(수요기관)', '지역제한',
       '010_001', '010_002', '010_003', '010_004', '010_005', '010_006',
       '010_007', '010_008', '010_009', '010_010', '020_001', '020_002',
       '020_003'],
      dtype='object')

In [26]:
Basic_X = organized_df.iloc[:,:10]
Basic_X.columns
Basic_Y = organized_df.iloc[:,10:]
Basic_Y.set_index(Basic_X['공고번호'], inplace=True)
print(Basic_Y.columns[:10])

Modified_X = normalized_df.iloc[:,:17]
Modified_X.columns
Modified_Y = normalized_df.iloc[:,17:]
Modified_Y.set_index(Modified_X['공고번호'], inplace=True)
Modified_Y.columns[:10]

Index(['010_001', '010_002', '010_003', '010_004', '010_005', '010_006',
       '010_007', '010_008', '010_009', '010_010'],
      dtype='object')


Index(['010_001', '010_002', '010_003', '010_004', '010_005', '010_006',
       '010_007', '010_008', '010_009', '010_010'],
      dtype='object')

In [27]:
Basic_X.to_csv('Combined_Basic_X_Range2.csv', index=False)
Basic_Y.to_csv('Combined_Basic_Y_Range2.csv', index=True)
Modified_X.to_csv('Combined_Modified_X_Range2.csv', index=False)
Modified_Y.to_csv('Combined_Modified_Y_Range2.csv', index=True)

In [10]:
# Maked randomized
def shuffle_specific_columns(df):
    """
    지정된 컬럼들만 독립적으로 셔플합니다.
    
    Parameters:
    -----------
    df : DataFrame
        원본 데이터프레임
    
    Returns:
    --------
    DataFrame
        지정된 컬럼들만 셔플된 데이터프레임
    """
    import numpy as np
    
    columns_to_shuffle = [
        '기초금액',
        'log_기초금액',
        'log_기초금액_norm',
        '기초금액_norm',
        'A값',
        'A값/기초금액_norm',
        '투찰률_norm',
        '참여업체수',
        'log_참여업체수',
        'log_참여업체수_norm',
        '예가범위',
        '공고구분표시',
        '발주처(수요기관)',
        '지역제한',
        '정답사정률(%)'
    ]
    
    shuffled_df = df.copy()
    
    # 지정된 컬럼만 셔플
    for col in columns_to_shuffle:
        if col in df.columns:  # 해당 컬럼이 존재하는 경우에만 셔플
            shuffled_df[col] = np.random.permutation(df[col].values)

    return shuffled_df

In [11]:
def debug_numeric_conversion(df, function_name=''):
    """
    DataFrame의 숫자형 변환 과정에서 발생할 수 있는 문제를 디버그합니다.
    
    Parameters:
    -----------
    df : DataFrame
        검사할 데이터프레임
    function_name : str
        어떤 함수에서 호출되었는지 표시 (선택사항)
    """
    import numpy as np
    
    if function_name:
        print(f"\n=== Debugging {function_name} ===\n")
    
    numeric_columns = ['기초금액', 'A값', '투찰률(%)', '참여업체수']
    
    for col in numeric_columns:
        try:
            print(f"\n📊 Analyzing column: {col}")
            print("-" * 80)
            
            # 1. 컬럼 존재 여부 확인
            if col not in df.columns:
                print(f"❌ Column {col} not found in DataFrame!")
                continue
                
            # 2. 데이터 타입 확인
            print(f"Current dtype: {df[col].dtype}")
            
            # 3. Unique 값 확인
            unique_vals = df[col].unique()
            print(f"\nNumber of unique values: {len(unique_vals)}")
            print("Sample of unique values:", unique_vals[:10])
            
            # 4. 문제될 수 있는 값들 검사
            non_numeric = []
            for val in df[col].unique():
                if isinstance(val, str) and not val.replace('.','').replace('-','').isdigit():
                    non_numeric.append(val)
            
            if non_numeric:
                print("\n⚠️ Non-numeric values found:")
                print(non_numeric)
                
                # 문제되는 값들의 위치와 해당 행의 모든 데이터 출력
                for prob_val in non_numeric:
                    prob_rows = df[df[col] == prob_val]
                    print(f"\n🔍 Rows containing '{prob_val}' in column '{col}':")
                    print(f"Found in {len(prob_rows)} rows")
                    
                    if len(prob_rows) > 0:
                        print("\nFirst few problematic rows:")
                        print("-" * 80)
                        for idx, row in prob_rows.head().iterrows():
                            print(f"\nRow Index: {idx}")
                            for column, value in row.items():
                                print(f"{column}: {value}")
                            print("-" * 80)
            
            # 5. 숫자형 변환 테스트
            try:
                pd.to_numeric(df[col], errors='raise')
                print("\n✅ Column can be safely converted to numeric")
            except Exception as e:
                print(f"\n❌ Numeric conversion error: {str(e)}")
            
        except Exception as e:
            print(f"\n❌ Error analyzing column {col}: {str(e)}")
            
    print("\n=== Debug Analysis Complete ===")

In [12]:
# 원본 데이터에 대해 실행
debug_numeric_conversion(raw_data, "Original Data")

# reorganize 후 데이터에 대해 실행
debug_numeric_conversion(raw_data_reorganized, "After Reorganization")


=== Debugging Original Data ===


📊 Analyzing column: 기초금액
--------------------------------------------------------------------------------
Current dtype: float64

Number of unique values: 3278
Sample of unique values: [1.509400e+07 1.500000e+08 3.730000e+07 1.900000e+07 2.299660e+08
 6.594367e+09 6.729021e+09 1.334160e+08 1.548300e+08 2.387300e+07]

✅ Column can be safely converted to numeric

📊 Analyzing column: A값
--------------------------------------------------------------------------------
Current dtype: float64

Number of unique values: 751
Sample of unique values: [0.00000000e+00 2.47017330e+07 1.62688524e+08 1.96173870e+07
 4.15545500e+06 7.25466600e+06 4.56396000e+06 8.31842500e+06
 7.31669900e+06 3.28207400e+06]

✅ Column can be safely converted to numeric

📊 Analyzing column: 투찰률(%)
--------------------------------------------------------------------------------
Current dtype: object

Number of unique values: 15
Sample of unique values: ['87.745' '90.0' '85.495' '86.745' 